## Votos Congresionales - SQL

Este cuaderno replica en SQL los principales procedimientos hechos en Python en el cuaderno `votos_congresionales.ipynb`. La primera parte está implementada directamente en SQL, utilizando el módulo `mysql` de Python que me permite conectarme a una base de datos de MySQL que previamente creé y donde cargué los datos de votaciones utilizados en esta prueba.

In [103]:
# Importar módulos
import pandas as pd
import mysql.connector
import json

Conexión con base de datos de MySQL

In [106]:
# Importar credenciales
with open('../src/config', 'r') as f:
    config = json.load(f)

# Crear conexión
cnx = mysql.connector.connect(**config)

# Crear cursor para ejecutar consultas en base de datos
cursor = cnx.cursor(dictionary=True)

### Análisis exploratorio y limpieza de los datos

Visualizar datos

In [40]:
query = (
    "SELECT * FROM `house-votes-84` LIMIT 4"
)

cursor.execute(query)
pd.DataFrame(cursor)

,party,handicapped_infants,water_project,budget_resolution,physicial_fee_freeze,el_salvador_aid,religious_groups_schools,anti_satellite_test_ban,aid_contras,mx_missile,immigration,synfuels_cutback,education_spending,superfund_right_to_sue,crime,duty_free_exports,export_admin_act_south_africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y


Número de observaciones

In [44]:
query = (
    "SELECT COUNT(*) observaciones FROM `house-votes-84`"
)

cursor.execute(query)
pd.DataFrame(cursor)

,observaciones
0,435


Estructura de los datos

In [41]:
query = (
    "DESCRIBE `house-votes-84`"
)

cursor.execute(query)
pd.DataFrame(cursor)

,Field,Type,Null,Key,Default,Extra
0,party,text,YES,,None,
1,handicapped_infants,text,YES,,None,
2,water_project,text,YES,,None,
3,budget_resolution,text,YES,,None,
4,physicial_fee_freeze,text,YES,,None,
5,el_salvador_aid,text,YES,,None,
6,religious_groups_schools,text,YES,,None,
7,anti_satellite_test_ban,text,YES,,None,
8,aid_contras,text,YES,,None,
9,mx_missile,text,YES,,None,


Validar que en la documentación se refieren a `?` cuando hablan de missing (resultado debe ser 12)

In [84]:
query = """
SELECT COUNT(*) observaciones 
FROM `house-votes-84`
WHERE handicapped_infants = '?'
"""

cursor.execute(query)
pd.DataFrame(cursor)

,observaciones
0,12


#### Análisis univariado

Proporción de votos por partido

In [75]:
query = """
SELECT republicans, 
       republicans / votings * 100 republicans_perc, 
       democrats, 
       democrats / votings * 100 democrats_perc
FROM (
    SELECT 
        SUM(CASE WHEN party = 'republican' THEN 1 ELSE 0 END) republicans,
        SUM(CASE WHEN party = 'democrat' THEN 1 ELSE 0 END) democrats, 
        COUNT(*) votings
    FROM `house-votes-84`
) party_count
"""

cursor.execute(query)
pd.DataFrame(cursor)

,republicans,republicans_perc,democrats,democrats_perc
0,168,38.6207,267,61.3793


Resultados de votaciones

In [94]:
query = "SELECT column_name FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'house-votes-84'"
cursor.execute(query)
columns = [row[0] for row in pd.DataFrame(cursor).values.tolist()]

In [95]:
for column in columns:
    query = f"""
    SELECT {column}, COUNT(*) votos
    FROM `house-votes-84`
    GROUP BY {column} 
    """

    cursor.execute(query)
    print(pd.DataFrame(cursor), end='\n\n')

  aid_contras  votos
0           n    178
1           y    242
2           ?     15

  anti_satellite_test_ban  votos
0                       n    182
1                       y    239
2                       ?     14

  budget_resolution  votos
0                 n    171
1                 y    253
2                 ?     11

  crime  votos
0     y    248
1     n    170
2     ?     17

  duty_free_exports  votos
0                 n    233
1                 y    174
2                 ?     28

  education_spending  votos
0                  y    171
1                  n    233
2                  ?     31

  el_salvador_aid  votos
0               y    212
1               ?     15
2               n    208

  export_admin_act_south_africa  votos
0                             y    269
1                             ?    104
2                             n     62

  handicapped_infants  votos
0                   n    236
1                   ?     12
2                   y    187

  immigration  

#### Análisis bivariado

In [102]:
for column in columns:
    query = f"""
    SELECT party, {column} voto, COUNT(*) votos
    FROM `house-votes-84`
    GROUP BY party, {column}
    ORDER BY party
    """

    cursor.execute(query)
    print(column, ":", sep="")
    print(pd.DataFrame(cursor), end='\n\n')

aid_contras:
        party voto  votos
0    democrat    ?      4
1    democrat    n     45
2    democrat    y    218
3  republican    ?     11
4  republican    n    133
5  republican    y     24

anti_satellite_test_ban:
        party voto  votos
0    democrat    ?      8
1    democrat    n     59
2    democrat    y    200
3  republican    ?      6
4  republican    n    123
5  republican    y     39

budget_resolution:
        party voto  votos
0    democrat    ?      7
1    democrat    n     29
2    democrat    y    231
3  republican    ?      4
4  republican    n    142
5  republican    y     22

crime:
        party voto  votos
0    democrat    ?     10
1    democrat    n    167
2    democrat    y     90
3  republican    ?      7
4  republican    n      3
5  republican    y    158

duty_free_exports:
        party voto  votos
0    democrat    ?     16
1    democrat    n     91
2    democrat    y    160
3  republican    ?     12
4  republican    n    142
5  republican    y     14

ed

### Modelación

Por restricción de tiempo y la complejidad que implica crear estos modelos en SQL, no intentaré resolver esta sección en SQL. Sin embargo, explicaré cuál sería la estrategia general para abordar este problema.

- Para codificar todos los valores de tipo string a numéricas, crearé una vista nueva con las misma observaciones pero utilizaré la cláusula `CASE` para reemplazar todos los valores.
- Para dividir los datos en conjuntos de prueba y validación, crearía una vista llamada validación que seleccione el 20% de las filas al azar (utilizando `LIM` equivalente al 20% del conteo de observaciones), utilizando la función `RAND()` e incluyendo un `seed` para garantizar que siempre tenga los mismos valores. Después crearía otra vista llamada entrenamiento que incluya los valores de la tabla `house-votes-84` que no estén contenidos en la vista validación.
- Para calcular las probabilidades `Pr(X=x|y)` que me permitan pronósticar el partido de cada observación/representante crearía una variable para calcular la probabilidad de cada combinación de `x` y `y` para los valores de la observación y las utilizaría en un statement que multiplique cada una de estas variables para encontrar la probabilidad posterior `Pr(Y|X)`, es decir la probabilidad de que el candidato sea republicano.

In [4]:
cnx.close()